In [1]:
# from multiprocessing import Pool, Manager
from tqdm import tqdm
from hashlib import md5

class NgramModel():
    def __init__(self):
        pass
    
    def load_data(self, train_data_file):
        self.train_data = []
        with open(train_data_file, 'r') as fin:
            for line in fin:
                self.train_data.append(["<s>"] + line.strip().split(" ") + ["</s>"])
        
                
    def unsmoothed_n_grams(self, n, num_thread=4):
        word_count = {}
        
        def ngrams(lst_words, n):
            if len(lst_words) < n:
                return 
            for i in range(len(lst_words) - n + 1):
                key = " ".join(lst_words[i:i+n])
                key = md5(key).hexdigest()
                word_count.setdefault(key, 0)
                word_count[key] += 1
                
        for lst_words in tqdm(self.train_data):
            ngrams(lst_words, n)
        
        return word_count

In [ ]:
model = NgramModel()

model.load_data("../data/comment_corpus_for_lm.train.txt")

unigram = model.unsmoothed_n_grams(1)
bigram = model.unsmoothed_n_grams(2)
trigram = model.unsmoothed_n_grams(3)

 22%|██▏       | 3692160/16898316 [01:22<05:09, 42618.55it/s]

In [ ]:
import cPickle as pkl

pkl.dump((unigram, bigram, trigram), open("./DQD_vocab.pkl", 'wb'))

## Test

In [ ]:
import cPickle as pkl

unigram, bigram, trigram = pkl.load(open("./DQD_vocab.pkl", 'rb'))

In [3]:
from hashlib import md5
from math import log

ROOT_PATH = "../../working/c2c_char_level/kgcVAE_with_BOW_loss/run1509882751/"

test_file = ROOT_PATH + "test.txt.words"
output_file = ROOT_PATH + "test.txt.words.ngram.prob"

V = vocab_size = len(unigram)

def get_prob(word_seq):
    """
    word_seq: ["<s>", ..., "</s>"]
    return: [prob, ...]
    """
    assert(len(word_seq) > 2)
    probs = []
    sum_prob = 0.0
    for i in range(0, len(word_seq)-2):
        tri_key = md5(" ".join(word_seq[i:i+3])).hexdigest()
        tri_count = trigram.get(tri_key, 0) + 1
        bi_key = md5(" ".join(word_seq[i:i+2])).hexdigest()
        bi_count = bigram.get(bi_key, 0) + V
        logprob = log(tri_count * 1.0 / bi_count)
        probs.append(logprob)
        sum_prob += logprob
    probs.append(sum_prob)
    return probs

with open(test_file, 'r') as fin:
    with open(output_file, 'w') as fout:
        for line in fin:
            word_seq = ["<s>"] + line.strip().split(" ") + ["</s>"]
            probs = get_prob(word_seq)
            print >> fout, "\t".join(map(str, probs))